In [ ]:
from stk_actor.agent import UnifiedSACPolicy
from stk_actor.wrappers import PreprocessObservationWrapper, StuckStopWrapper, SkipFirstNStepsWrapper
from stable_baselines3 import PPO, A2C
from stable_baselines3.common.env_util import make_vec_env
from pystk2_gymnasium import AgentSpec

import torch

path = 'stk_actor/trained_agents/'
agent = 'normed_behavioral_cloning_num5'

tracks = [
    'abyss',
    'black_forest',
    'candela_city',
    'cocoa_temple',
    'cornfield_crossing',
    'fortmagma',
    'gran_paradiso_island',
    'hacienda',
    'lighthouse',
    'mines',
    'minigolf',
    'olivermath',
    'ravenbridge_mansion',
    'sandtrack',
    'scotland',
    'snowmountain',
    'snowtuxpeak',
    'stk_enterprise',
    'volcano_island',
    'xr591',
    'zengarden',

# # #   ==================   #

#     'fortmagma',
#     'ravenbridge_mansion',
#     'snowmountain',
#     'cocoa_temple',
#     'sandtrack',    
#     'scotland', 
#     'stk_enterprise',
#     'volcano_island', # 1104
#     'xr591', # 864   
]

karts = [12]
n_envs = len(tracks)*len(karts)

print('making', n_envs, 'environments')
vec_env = make_vec_env(
    "supertuxkart/flattened_multidiscrete-v0",
    # seed=12,
    n_envs=n_envs, 
    wrapper_class=lambda x : (
        SkipFirstNStepsWrapper(
            StuckStopWrapper(
                PreprocessObservationWrapper(x, ret_dict=False, norm=True, agent_name=agent),
                n=92,
            ), 
            n=20,
        )
    ), 
    env_kwargs={
    'render_mode':None, 'agent':AgentSpec(use_ai=False, name="walid"), #'track':'minigolf', 
    'laps':1,
    'difficulty':2, 
    'num_kart':12, #'difficulty':0
})

ix = 0
for num_kart in enumerate(karts):
    for track in enumerate(tracks):
        venv = vec_env.envs[ix]
        venv.env.default_track = track
        venv.env.num_kart = num_kart
        print(ix, track, )
        ix+=1

net_arch=[1024,1024,1024]
activation_fn=torch.nn.Tanh
filename = path+f'{agent}/statedict'

action_dims = [space.n for space in vec_env.action_space]
unified_policy = UnifiedSACPolicy(
    vec_env.observation_space, 
    action_dims, 
    net_arch=net_arch, 
    activation_fn=activation_fn
)
unified_policy.load_state_dict(torch.load(filename, map_location='cpu'))


making 21 environments
..:: Antarctica Rendering Engine 2.0 ::..


/var/folders/5b/wsyz4crx22bg_7tpsgbw3tzw0000gn/T/ipykernel_40078/3042624347.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.mean = torch.load(path+f'{agent}/buffer

0 (0, 'abyss')
1 (1, 'black_forest')
2 (2, 'candela_city')
3 (3, 'cocoa_temple')
4 (4, 'cornfield_crossing')
5 (5, 'fortmagma')
6 (6, 'gran_paradiso_island')
7 (7, 'hacienda')
8 (8, 'lighthouse')
9 (9, 'mines')
10 (10, 'minigolf')
11 (11, 'olivermath')
12 (12, 'ravenbridge_mansion')
13 (13, 'sandtrack')
14 (14, 'scotland')
15 (15, 'snowmountain')
16 (16, 'snowtuxpeak')
17 (17, 'stk_enterprise')
18 (18, 'volcano_island')
19 (19, 'xr591')
20 (20, 'zengarden')


/var/folders/5b/wsyz4crx22bg_7tpsgbw3tzw0000gn/T/ipykernel_40078/3042624347.py:327: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  unified_policy.load_state_dict(torch.load(f

<All keys matched successfully>

In [ ]:
steps = [(
    # 2048,
    10*n_envs,
    300_000,
)]
for n_steps, total_timesteps in steps:
    # model = PPO(
    #     "MlpPolicy", 
    #     vec_env, 
    #     verbose=1, 
    #     policy_kwargs = dict(net_arch=net_arch, activation_fn=activation_fn,),
    #     device='cpu',
    #     learning_rate=0.0001,
    #     batch_size=128,
    #     n_epochs=100,
    #     n_steps=n_steps,
    #     tensorboard_log="./outputs/",
    #     clip_range=0.2,
    # )
    model = A2C(
        "MlpPolicy", 
        vec_env, 
        verbose=1, 
        policy_kwargs = dict(net_arch=net_arch, activation_fn=activation_fn,),
        device='cpu',
        learning_rate=0.001,
        n_steps=n_steps,
        tensorboard_log="./outputs/",
        use_rms_prop=False,
        normalize_advantage=True,
    )
    print('DOING', n_steps, total_timesteps)
    model.policy.load_state_dict(unified_policy.shared.state_dict())
    model.learn(total_timesteps=total_timesteps, progress_bar=True, log_interval=1)


Using cpu device
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
..:: Antarctica Rendering Engine 2.0 ::..
DOING 210 300000
Logging to ./outputs/A2C_10


Output()

## Saving

In [ ]:
# unified_policy.shared.load_state_dict(model.policy.state_dict())
# torch.save(unified_policy.state_dict(), 'stk_actor/trained_agents/normed_a2c_num5_best/statedict')

In [ ]:
# unified_policy.shared.load_state_dict(model.policy.state_dict())
# torch.save(unified_policy.state_dict(), 'stk_actor/trained_agents/normed_ppo_num5_best/statedict')